In [1]:
import pandas as pd
from params import *
from data import *
import pickle
from models import *
import os
from torch import nn
from torch.optim import Adam
from train import Trainer
from eval import *
from metrics import *
from sklearn.manifold import TSNE
from encoding import *

In [ ]:
pth = 'data/big_data/kinetics400_5per/train/'
pth_to_save_images = 'data/big_data/kinetics400_5per/devided/images/'
pth_to_save_audio = 'data/big_data/kinetics400_5per/devided/audios/'

In [ ]:
import cv2
from pydub import AudioSegment
import os

c = 0

def extract_frame_and_audio(video_path, image_output_path, audio_output_path):
    """
    Извлекает первый кадр из видео и аудиодорожку, сохраняя их в указанные пути.
    Использует OpenCV для извлечения кадра и pydub для извлечения аудио.
    Убедитесь, что пути к изображению имеют правильное расширение (например, .jpg или .png).

    Args:
        video_path (str): Путь к исходному MP4 файлу.
        image_output_path (str): Путь для сохранения первого кадра (изображения). Должен заканчиваться на .jpg или .png.
        audio_output_path (str): Путь для сохранения аудиодорожки (WAV файл).
    """
    # Извлечение первого кадра с помощью OpenCV
    cap = cv2.VideoCapture(video_path)
    ret, frame = cap.read()
    if ret:
        # Убедимся, что путь к изображению имеет правильное расширение
        if not (image_output_path.lower().endswith('.jpg') or image_output_path.lower().endswith('.png')):
            raise ValueError("Путь к изображению должен заканчиваться на .jpg или .png")
        cv2.imwrite(image_output_path, frame)
    else:
        raise ValueError(f"Не удалось прочитать кадр из видео: {video_path}")
    cap.release()

    # Извлечение аудио с помощью pydub
    try:
        audio = AudioSegment.from_file(video_path, format="mp4")
        audio.export(audio_output_path, format="wav")
    except:
        c += 1

In [ ]:
imgs = {
    OBJECT1_COLUMN_NAME: [],
    OBJECT2_COLUMN_NAME: []
}
for t in tqdm(os.listdir(pth_to_save_images)):
    for f in os.listdir(pth_to_save_images + t):
        imgs[OBJECT1_COLUMN_NAME].append(pth_to_save_images + t + '/' + f)
        imgs[OBJECT2_COLUMN_NAME].append(t)

In [ ]:
df_image = pd.DataFrame.from_dict(imgs)

In [ ]:
df_image[TYPE1_COLUMN_NAME] = TYPE_IMAGE
df_image[TYPE2_COLUMN_NAME] = TYPE_TEXT

In [ ]:
audios = {
    OBJECT1_COLUMN_NAME: [],
    OBJECT2_COLUMN_NAME: []
}
for t in tqdm(os.listdir(pth_to_save_audio)):
    for f in os.listdir(pth_to_save_audio + t):
        audios[OBJECT1_COLUMN_NAME].append(pth_to_save_audio + t + '/' + f)
        audios[OBJECT2_COLUMN_NAME].append(t)

In [ ]:
df_audio = pd.DataFrame.from_dict(audios)

In [ ]:
df_audio[TYPE1_COLUMN_NAME] = TYPE_AUDIO
df_audio[TYPE2_COLUMN_NAME] = TYPE_TEXT

In [ ]:
df = pd.concat([df_image, df_audio], axis=0, ignore_index=True)

In [ ]:
initial = InitialDataset(df)